In [22]:
from transformers import pipeline
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

pipe = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", framework="pt", device=device)

In [3]:
from typing import AnyStr
import os
os.chdir("..")
os.chdir("src")
from data_cleaning_pandas import DataCleaning
os.chdir("..")

In [4]:
cleaning_pipeline = DataCleaning()
df = cleaning_pipeline.run()

2024-04-01 08:49:58 - INFO - Data loaded!
2024-04-01 08:49:58 - INFO - Null values cleaned!
/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/src/data_cleaning_pandas.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["ANO_LICITACAO"] = df_final["ANO_LICITACAO"].replace(
2024-04-01 08:49:58 - INFO - Data types asserted!
2024-04-01 08:49:58 - INFO - Full data cleaned!


In [4]:
df

,CD_ORGAO,NM_ORGAO,ANO_LICITACAO,DS_OBJETO,VL_LICITACAO,DT_HOMOLOGACAO,VL_HOMOLOGADO
0,47500,PM DE ESTÂNCIA VELHA,2017,"CONTRATAÇÃO, EXCLUSIVA PARA EMPRESAS BENEFICIÁ...",72691.16,2018-01-17,72691.16
1,75300,PM DE DERRUBADAS,2017,Aquisição de alimentos.,65808.91,2017-01-23,65808.91
2,88027,PM DE CAPÃO DO CIPÓ,2017,AQUISIÇÃO DE COMBUSTIVEIS PARA MANUTENÇÃO DA F...,1058588.00,2017-02-01,1112500.00
3,88039,PM DE PINHAL DA SERRA,2017,O objeto da presente licitação é o registro de...,1361100.00,2017-01-27,1361100.00
4,69900,PM DE PINHAL,2017,AQUISIÇÃO DE MATERIAIS DE LIMPEZA,28942.06,2017-01-25,28942.06
...,...,...,...,...,...,...,...
305595,40600,PM DE ALVORADA,2021,Registro de preço para aquisição de equipament...,3726826.37,2022-12-26,2407569.55
305596,50100,PM DE IJUÍ,2021,Credenciamento de pessoas jurídicas e/ou físic...,132015.00,NaT,132015.00
305597,55600,PM DE RIO GRANDE,2021,Exames e avaliações de Ecografias,249900.00,NaT,249900.00
305598,63400,PM DE CAPÃO DA CANOA,2021,Registro de Preços para contratação de empresa...,1736650.00,2021-09-24,664900.00


In [18]:
df_trial = df.head(5)

In [43]:
inputs = pipe.tokenizer.encode(df_trial["DS_OBJETO"].values[0])

In [50]:
with torch.no_grad():
    logits = pipe(**inputs).logits

TypeError: <transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x74cd31f46c50> argument after ** must be a mapping, not list

In [47]:
output = pipe.model

AttributeError: 'DebertaV2ForSequenceClassification' object has no attribute 'predict'

In [46]:
output = pipe.model.predict(inputs)  # device = "cuda:0" or "cpu"

prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

TypeError: list indices must be integers or slices, not tuple

In [ ]:
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

In [ ]:
output = model(inputs["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

In [39]:
zeroshot_topic_list = ["vehicles", "construction", "health/hospital", "education", "food"]

In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", problem_type="zero-shot-classification")

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")


In [17]:
tokenizer.encode(df_trial["DS_OBJETO"].values[0])

[1,
 121147,
 491,
 61664,
 262,
 260,
 117684,
 400,
 34419,
 20826,
 260,
 137273,
 47448,
 172064,
 179753,
 3615,
 142961,
 8909,
 48408,
 645,
 221170,
 262,
 1451,
 21265,
 218621,
 260,
 195821,
 81132,
 7330,
 20826,
 299,
 260,
 179014,
 61664,
 1451,
 111362,
 4025,
 3808,
 1451,
 25313,
 156658,
 491,
 37574,
 61664,
 276,
 25313,
 204257,
 998,
 61664,
 82211,
 155926,
 74973,
 260,
 114083,
 11297,
 43058,
 416,
 364,
 81081,
 404,
 28914,
 149072,
 106375,
 400,
 8909,
 70614,
 30706,
 80820,
 1451,
 260,
 98544,
 48327,
 416,
 260,
 136159,
 416,
 260,
 179014,
 61664,
 1451,
 111362,
 4025,
 3808,
 1451,
 459,
 121428,
 95799,
 1451,
 6281,
 179069,
 92666,
 118114,
 82211,
 155926,
 74973,
 260,
 114083,
 11297,
 43058,
 416,
 364,
 81081,
 404,
 28914,
 149072,
 106375,
 400,
 262,
 8909,
 70614,
 30706,
 80820,
 1451,
 260,
 98544,
 48327,
 416,
 260,
 136159,
 416,
 8909,
 70614,
 30706,
 80820,
 1451,
 6709,
 5929,
 6358,
 262,
 4678,
 372,
 111168,
 1451,
 34594,


In [5]:
candidate_labels = ["vehicles", "construction", "health/hospital", "education", "food"]


In [8]:
import torch

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

premise = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
hypothesis = "Emmanuel Macron is the President of France"

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

In [7]:
from sentence_transformers import SentenceTransformer, util

In [11]:
output_total = []

for obj in df_trial["DS_OBJETO"].values:
    output = pipe(obj, candidate_labels, multi_label=False)
    output_total.append(output["labels"][0])

AttributeError: 'dict' object has no attribute 'to'

In [43]:
output_total

['education', 'food', 'food', 'vehicles', 'construction']

In [44]:
df_trial["label"] = output_total

/tmp/ipykernel_11625/3031190406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trial["label"] = output_total


In [45]:
df_trial

,CD_ORGAO,NM_ORGAO,ANO_LICITACAO,DS_OBJETO,VL_LICITACAO,DT_HOMOLOGACAO,VL_HOMOLOGADO,label
0,47500,PM DE ESTÂNCIA VELHA,2017,"CONTRATAÇÃO, EXCLUSIVA PARA EMPRESAS BENEFICIÁ...",72691.16,2018-01-17,72691.16,education
1,75300,PM DE DERRUBADAS,2017,Aquisição de alimentos.,65808.91,2017-01-23,65808.91,food
2,88027,PM DE CAPÃO DO CIPÓ,2017,AQUISIÇÃO DE COMBUSTIVEIS PARA MANUTENÇÃO DA F...,1058588.00,2017-02-01,1112500.00,food
3,88039,PM DE PINHAL DA SERRA,2017,O objeto da presente licitação é o registro de...,1361100.00,2017-01-27,1361100.00,vehicles
4,69900,PM DE PINHAL,2017,AQUISIÇÃO DE MATERIAIS DE LIMPEZA,28942.06,2017-01-25,28942.06,construction


In [25]:
import pandas as pd

In [26]:
for item in output_total:
    # Adicionar o primeiro label de cada item à lista
    label = item['labels'][0]
    # Criar um DataFrame temporário para o item atual
    temp_df = pd.DataFrame({'sequence': [item['sequence']], 'label': [label]})
    # Adicionar o DataFrame temporário ao DataFrame principal
    df_trial = pd.concat([df_trial, temp_df])


In [27]:
df_trial

,CD_ORGAO,NM_ORGAO,ANO_LICITACAO,DS_OBJETO,VL_LICITACAO,DT_HOMOLOGACAO,VL_HOMOLOGADO,sequence,label
0,47500.0,PM DE ESTÂNCIA VELHA,2017.0,"CONTRATAÇÃO, EXCLUSIVA PARA EMPRESAS BENEFICIÁ...",72691.16,2018-01-17,72691.16,NaN,NaN
1,75300.0,PM DE DERRUBADAS,2017.0,Aquisição de alimentos.,65808.91,2017-01-23,65808.91,NaN,NaN
2,88027.0,PM DE CAPÃO DO CIPÓ,2017.0,AQUISIÇÃO DE COMBUSTIVEIS PARA MANUTENÇÃO DA F...,1058588.00,2017-02-01,1112500.00,NaN,NaN
3,88039.0,PM DE PINHAL DA SERRA,2017.0,O objeto da presente licitação é o registro de...,1361100.00,2017-01-27,1361100.00,NaN,NaN
4,69900.0,PM DE PINHAL,2017.0,AQUISIÇÃO DE MATERIAIS DE LIMPEZA,28942.06,2017-01-25,28942.06,NaN,NaN
...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,PREGÃO PRESENCIAL PARA AQUISIÇÃO DE GÊNEROS AL...,education
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,AQUISIÇÃO DE MATERIAIS DE LIMPEZA E PRODUTOS D...,health/hospital
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,CONTRATAÇÃO DE SERVIÇOS DE LAVAGEM DE VEÍCULOS...,vehicles
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,CONTRATAÇÃO DE EMPRESA OU ASSOCIAÇÃO PARA REAL...,construction


In [24]:
df_trial["label"] = [output_total[0] for output_total in df_trial["labels"]]

KeyError: 'labels'

In [17]:
output_total

[{'sequence': 'CONTRATAÇÃO, EXCLUSIVA PARA EMPRESAS BENEFICIÁRIAS DA LC 123/2006, DE EMPRESA ESPECIALIZADA PARA A PRESTAÇÃO DE SERVIÇOS DE DESINSETIZAÇÃO/ DESRATIZAÇÃO NAS ESCOLAS MUNICIPAIS E PRÉDIOS ADMINISTRATIVOS DA SECRETARIA DE EDUCAÇÃO E CULTURA E  PRESTAÇÃO DE SERVIÇOS DE LIMPEZA DE RESERVATÓRIOS NAS ESCOLAS MUNICIPAIS E PRÉDIOS ADMINISTRATIVOS, DA SECRETARIA DE EDUCAÇÃO E CULTURA E DA SECRETARIA DE SAÚDE, NA CIDADE DE ESTÂNCIA VELHA-RS, PARA O ANO DE 2018.',
  'labels': ['education',
   'health/hospital',
   'construction',
   'vehicles',
   'food'],
  'scores': [0.9850495457649231,
   0.006502756383270025,
   0.0039464253932237625,
   0.0027769836597144604,
   0.0017243162728846073]},
 {'sequence': 'Aquisição de alimentos.',
  'labels': ['food',
   'vehicles',
   'construction',
   'health/hospital',
   'education'],
  'scores': [0.8377798795700073,
   0.050969332456588745,
   0.04714931175112724,
   0.045277539640665054,
   0.01882397010922432]},
 {'sequence': 'AQUISIÇÃO DE 

In [33]:
sequence_to_classify = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = pipe(sequence_to_classify, candidate_labels, multi_label=False)

In [35]:
output['labels']

['politics', 'economy', 'entertainment', 'environment']

In [4]:
# precisa instalar o pytorch antes, pro meu caso foi o comando abaixo (mas no readme nao precisa passar o comando)
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 